In [34]:
# Import dependencies
import pandas as pd

import requests
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [35]:
# read in csv
all_vehicles = pd.read_csv("all_vehicles_ml.csv")

In [36]:
all_vehicles.head()

,Unnamed: 0,month,hour,year,day,count,date,maxtemp,atemp,mintemp,...,maxwind,awind,minwind,prec,day_of_week,vehicle_type,vehicle_encoded,hour_encoded,month_encoded,count_encoded
0,0,1,0,2019,1,342,2019-01-01,46,41,36,...,21,11,0,0,1,scooter,0,0,0,0
1,1,1,0,2019,2,79,2019-01-02,41,38,37,...,15,10,8,1,2,scooter,0,0,0,0
2,2,1,0,2019,3,16,2019-01-03,52,40,31,...,13,7,0,1,3,scooter,0,0,0,0
3,3,1,0,2019,4,82,2019-01-04,67,46,30,...,10,3,0,1,4,scooter,0,0,0,0
4,4,1,0,2019,5,188,2019-01-05,73,49,32,...,13,4,0,0,5,scooter,0,0,0,0


In [37]:
# get only the columns that are needed for our model and app
all_vehicles = all_vehicles[['date','day', 'month','year','hour',
                             'day_of_week','atemp','ahum','awind',
                             'prec','vehicle_encoded','hour_encoded',
                             'month_encoded','count_encoded','count']]

In [38]:
all_vehicles.head()

,date,day,month,year,hour,day_of_week,atemp,ahum,awind,prec,vehicle_encoded,hour_encoded,month_encoded,count_encoded,count
0,2019-01-01,1,1,2019,0,1,41,83,11,0,0,0,0,0,342
1,2019-01-02,2,1,2019,0,2,38,92,10,1,0,0,0,0,79
2,2019-01-03,3,1,2019,0,3,40,85,7,1,0,0,0,0,16
3,2019-01-04,4,1,2019,0,4,46,66,3,1,0,0,0,0,82
4,2019-01-05,5,1,2019,0,5,49,66,4,0,0,0,0,0,188


In [27]:
# create a dataframe for vehicle types 
vehicle_dataframe = pd.DataFrame({
                    "vehicle_encoded":[0,1],
                    "vehicle_type":['scooter','bicycle']
})
vehicle_dataframe

,vehicle_encoded,vehicle_type
0,0,scooter
1,1,bicycle


In [25]:
# create a dataframe mapping hour groups
hour_dataframe = pd.DataFrame({
                    "hour_encoded":[0,1,2,3,4,5],
                    "hour_group":['Midnight - 3am','4am - 7am','8am - 11am',
                                 'Noon - 3pm','4pm - 7pm','8pm - 11pm']})
hour_dataframe

,hour_encoded,hour_group
0,0,Midnight - 3am
1,1,4am - 7am
2,2,8am - 11am
3,3,Noon - 3pm
4,4,4pm - 7pm
5,5,8pm - 11pm


In [30]:
# create a dataframe mapping month groups
month_dataframe = pd.DataFrame({
                    "month_encoded": [0,1,2,3],
                    "month_group":['January - March', 'April - June',
                                  'July - September','October - December']
})
month_dataframe

,month encoded,month_group
0,0,January - March
1,1,April - June
2,2,July - September
3,3,October - December


In [59]:
# create a dataframe mapping count groups
count_dataframe = pd.DataFrame({
                    "count_encoded": [0,1,2,3,4,5,6,7,8,9,10],
                    "count_grouped": ['0 - 435', '436 - 869', '870 - 1304',
                                    '1305 - 1738','1739 - 2171', '2172 - 2607',
                                    '2508 - 3011', '3012 - 3476', '3478 - 3892',
                                    '3893 - 4334','4335 - 4780']
})
count_dataframe

,count_encoded,count_grouped
0,0,0 - 435
1,1,436 - 869
2,2,870 - 1304
3,3,1305 - 1738
4,4,1739 - 2171
5,5,2172 - 2607
6,6,2508 - 3011
7,7,3012 - 3476
8,8,3478 - 3892
9,9,3893 - 4334


In [60]:
# connect to local postgres
rds_connection_string = "postgres:<password>@localhost:5432/micromobility"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [61]:
# check if table exists in animals database
engine.table_names()

['all_vehicles', 'vehicles', 'hours', 'months', 'counts']

In [63]:
# load dataframes to postgres database
all_vehicles.to_sql(name = 'all_vehicles', con = engine, if_exists="replace", index=False)
vehicle_dataframe.to_sql(name = 'vehicles', con = engine, if_exists="replace", index=False)
hour_dataframe.to_sql(name = 'hours', con = engine, if_exists="replace", index=False)
month_dataframe.to_sql(name = 'months', con = engine, if_exists="replace", index=False)
count_dataframe.to_sql(name = 'counts', con = engine, if_exists="replace", index=False)